# SodiumChannel

## 电生理特性

钠离子通道是神经元中不可或缺的兴奋性通道，它在动作电位的启动和上升相中发挥着核心作用。在膜电位达到一定阈值时，钠通道快速激活并造成钠离子内流，导致膜电位迅速去极化。

钠通道具有典型的双门控机制：

- 激活门：快速响应膜电位的变化，决定通道是否打开。
- 失活门：稍慢地响应膜电位，关闭通道使其失活。

这种双门控结构使钠通道呈现出快速激活与快速失活的动力学行为，是动作电位快速起落的根本机制。

钠通道对膜电位变化极为敏感，动力学特性决定了其对神经元兴奋性的调节作用。在经典的 HH 模型中，钠电流通常表示为：

$$
I_{Na} = \bar{g}_{Na} \cdot m^3 h \cdot (V - E_{Na})
$$

其中 $m$ 为激活变量，$h$ 为失活变量，$g_{Na}$ 是最大电导，$E_{Na}$ 是钠离子的反转电位。

## 建模实现

在 `braincell` 中，我们通过继承 `SodiumChannel` 类来实现钠离子通道的建模。该类本质上也一样是 `Channel` 的子类。

与前面介绍的钙通道、钾通道一致，我们只需定义相应的接口就可以完成建模。

下面我们依旧来看一个具体的例子：`INa_p3q_markov`。

值得一提的是，这里我们采用了不同的通道命名方式。 `INa_p3q_markov` 是根据公式结构来命名的，表示电流为 $p^3 q$，并采用 Markov 动力学形式。我们绝大多数的通道都是依靠作者与文章年份和公式结构这两种方法来命名的。

`INa_p3q_markov` 是一个采用 Markov 链建模形式的钠通道模型，其中电流由激活变量 $p$ 和失活变量 $q$ 控制：

$$
\begin{aligned}
I_{\mathrm{Na}} &= g_{\mathrm{max}} \cdot p^3 \cdot q \cdot (V - E_{Na}) \\\\
\frac{dp}{dt} &= \phi \cdot (\alpha_p (1 - p) - \beta_p p) \\\\
\frac{dq}{dt} &= \phi \cdot (\alpha_q (1 - q) - \beta_q q)
\end{aligned}
$$

其中：
- `p` 是激活变量
- `q` 是失活变量
- `φ` 是温度因子
- $g_{max}$ 是最大电导
- $E_{Na}$ 是钠离子反转电位

具体代码如下：

```python
class INa_p3q_markov(SodiumChannel):
    def __init__(
        self,
        size,
        g_max=90. * (u.mS / u.cm ** 2),
        phi=1.,
        name=None,
    ):
        super().__init__(size=size, name=name)
        self.phi = brainstate.init.param(phi, self.varshape, allow_none=False)
        self.g_max = brainstate.init.param(g_max, self.varshape, allow_none=False)

    def init_state(self, V, Na: IonInfo, batch_size=None):
        self.p = DiffEqState(brainstate.init.param(u.math.zeros, self.varshape, batch_size))
        self.q = DiffEqState(brainstate.init.param(u.math.zeros, self.varshape, batch_size))

    def reset_state(self, V, Na: IonInfo, batch_size=None):
        self.p.value = self.f_p_alpha(V) / (self.f_p_alpha(V) + self.f_p_beta(V))
        self.q.value = self.f_q_alpha(V) / (self.f_q_alpha(V) + self.f_q_beta(V))

    def compute_derivative(self, V, Na: IonInfo):
        self.p.derivative = self.phi * (self.f_p_alpha(V) * (1. - self.p.value) - self.f_p_beta(V) * self.p.value) / u.ms
        self.q.derivative = self.phi * (self.f_q_alpha(V) * (1. - self.q.value) - self.f_q_beta(V) * self.q.value) / u.ms

    def current(self, V, Na: IonInfo):
        return self.g_max * self.p.value ** 3 * self.q.value * (Na.E - V)

    def f_p_alpha(self, V):
        raise NotImplementedError

    def f_p_beta(self, V):
        raise NotImplementedError

    def f_q_alpha(self, V):
        raise NotImplementedError

    def f_q_beta(self, V):
        raise NotImplementedError
```

与其他两种离子通道完全相同，我们的钠通道建模依旧保持了高度灵活性。
